In [11]:
import requests
from time import sleep
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException


In [8]:
chrome_options = Options()
base_url = "https://championtraveler.com/flights/"

In [9]:
df=pd.read_csv("clean dataframe.csv")
df['Price'] = np.nan

In [10]:
sources=df['Origin_IATA'].unique().tolist()
prices=[]

In [25]:
### here we are extracting the flight_price for every pair (origin-destination) in the dataframe 
### after aמ extensive browsing on the average flight price between two airports, I found this website.
### Luckily it is structured in a fixed way, and finding a flight price was pretty easy

for orig in sources:
    filtered_df = df[df["Origin_IATA"] == orig]
    destinations = filtered_df["Destination_IATA"].unique().tolist()
    driver = webdriver.Chrome(options=chrome_options)
    for dest in destinations:
        if orig==dest:
            continue    
        url=base_url+orig+'-to-'+dest+'/'
        driver.get(url)
        try:
            elements = driver.find_elements(By.TAG_NAME, "strong")
            if len(elements) >= 2:
                element_text = elements[1].text.strip()
                words = element_text.split(" ")
                price = words[4]
            else:
                price = np.nan
        except NoSuchElementException:
            price = np.nan    
        
        df.loc[(df["Origin_IATA"] == orig) & (df["Destination_IATA"] == dest), 'Price'] = price
    sleep(15)
    driver.quit()
df.to_csv('dataframe with prices2.csv', index=False)    

In [13]:
df=pd.read_csv('dataframe with prices.csv')
df

,Origin,Origin_IATA,Flight_duration,Destination,Destination_IATA,Flights_per_day,distance (km),aircraft type,Operating airlines,Price
0,Amsterdam,AMS,80,Copenhagen,CPH,14,393,"['319', '321']",6,$122
1,Amsterdam,AMS,75,London,LHR,13,230,"['320', '767', '321', '319']",5,$99
2,Amsterdam,AMS,100,Dublin,DUB,12,465,['320'],1,$102
3,Amsterdam,AMS,130,Barcelona,BCN,13,770,"['73H', '73W']",5,$124
4,Amsterdam,AMS,65,London,LCY,10,208,['AR8'],4,$134
...,...,...,...,...,...,...,...,...,...,...
4264,Lyon,LYS,110,Batna,BLJ,1,691,['738'],1,$276
4265,Lyon,LYS,120,Biskra,BSK,1,755,['738'],1,$273
4266,Lyon,LYS,160,Gothenburg,GOT,1,878,['ER4'],2,$153
4267,Lyon,LYS,155,Oujda,OUD,1,839,['73H'],1,$409


In [30]:
df['Price'].isna().sum() # make sure it's reasonable number (it's 125)
df.dropna(subset='Price', inplace=True)
df['Price'] = df['Price'].str.replace('$', '',regex=False).astype(int)

In [15]:
df.reset_index(drop=True, inplace=True)
df.to_csv('final_df.csv', index=False)

In [16]:
df

,Origin,Origin_IATA,Flight_duration,Destination,Destination_IATA,Flights_per_day,distance (km),aircraft type,Operating airlines,Price
0,Amsterdam,AMS,80,Copenhagen,CPH,14,393,"['319', '321']",6,122
1,Amsterdam,AMS,75,London,LHR,13,230,"['320', '767', '321', '319']",5,99
2,Amsterdam,AMS,100,Dublin,DUB,12,465,['320'],1,102
3,Amsterdam,AMS,130,Barcelona,BCN,13,770,"['73H', '73W']",5,124
4,Amsterdam,AMS,65,London,LCY,10,208,['AR8'],4,134
...,...,...,...,...,...,...,...,...,...,...
4139,Lyon,LYS,110,Batna,BLJ,1,691,['738'],1,276
4140,Lyon,LYS,120,Biskra,BSK,1,755,['738'],1,273
4141,Lyon,LYS,160,Gothenburg,GOT,1,878,['ER4'],2,153
4142,Lyon,LYS,155,Oujda,OUD,1,839,['73H'],1,409


### Let's Make It A Numeric Dataframe
**Now, let's convert the data frame to a data frame with only numeric values, and then check 
In this section, we performed categorical data transformation by converting it into a numeric representation.
The motivation behind this transformation is to make the data understandable for machine learning algorithms, 
and look for correlation between the data frame's features.**


In [17]:
### This website has a table that divides the aircraft codes into 2 types: "Boeing" and "Airbus". 
### We will use this division to produce two numerical columns ("is_Boeing" and "is_Airbus") 
### and check the relationship between the type of aircraft and the flight price
### in general, an aircraft code starting with 'A' or '3' is an Airbus,
### and a code starting with 'B', 'E', or '7' is a Boeing. Let's create an auxiliary function

def categorize_aircraft_type(aircraft):
    is_boeing = 0
    is_airbus = 0
    
    for code in aircraft:
        if code.startswith('A') or code.startswith('3'):
            is_airbus = 1
        elif code.startswith('B') or code.startswith('E') or code.startswith('7'):
            is_boeing = 1
    
    return is_boeing, is_airbus

In [19]:
df.dropna(subset="aircraft type", inplace=True)
df[['is_Boeing', 'is_Airbus']] = df['aircraft type'].apply(categorize_aircraft_type).apply(pd.Series)
### removing all the categorical columns
columns_to_drop=['Origin','Origin_IATA','Destination','Destination_IATA','aircraft type']
df = df.drop(columns_to_drop, axis=1)
df=df.query('is_Boeing == 1 or is_Airbus == 1')

In [28]:
df.reset_index()
df.to_csv("numeric_df.csv", index=False)

,index,Flight_duration,Flights_per_day,distance (km),Operating airlines,Price,is_Boeing,is_Airbus
0,0,80,14,393,6,122,0,1
1,1,75,13,230,5,99,1,1
2,2,100,12,465,1,102,0,1
3,3,130,13,770,5,124,1,1
4,4,65,10,208,4,134,0,1
...,...,...,...,...,...,...,...,...
3860,4139,110,1,691,1,276,1,1
3861,4140,120,1,755,1,273,1,1
3862,4141,160,1,878,2,153,1,0
3863,4142,155,1,839,1,409,1,1
